In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --- Helper Functions ---

def preprocess_price_data(df):
    """Fill NaN values and missing dates with previous trading day's prices."""
    df = df.ffill()
    date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    df = df.reindex(date_range, method='ffill')
    return df

def load_price_data(file_path, sheet_name='price', rebal_date='2025-05-16'):
    """Load price data from Excel with DD-MM-YYYY date columns and filter for valid returns."""
    df = pd.read_excel(file_path, sheet_name=sheet_name, index_col=0, parse_dates=False)
    df = df.transpose()
    try:
        df.index = pd.to_datetime(df.index, format='%d-%m-%Y', errors='coerce')
    except Exception as e:
        print(f"Error converting dates: {e}")
        raise
    if df.index.isna().any():
        print("Warning: Some dates could not be parsed")
        raise
    df = df.replace('#NA', np.nan)
    df = preprocess_price_data(df)
    if df.isna().any().any():
        print(f"Warning: {df.isna().sum().sum()} NaN values remain after preprocessing")
    
    rebal_date = pd.to_datetime(rebal_date)
    m1 = df.index[df.index <= rebal_date][-1]
    m7 = df.index[df.index <= rebal_date - timedelta(days=180)][-1]
    m13 = df.index[df.index <= rebal_date - timedelta(days=360)][-1]
    key_dates = [m1, m7, m13]
    valid_stocks = df.loc[key_dates].notna().all(axis=0)
    df = df.loc[:, valid_stocks]
    
    print(f"Stocks with non-NaN prices on {key_dates}: {len(df.columns)}")
    return df

def load_ffmc_data(file_path, sheet_name='maktcap'):
    """Load free-float market cap data from Excel."""
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df['Date'] = pd.to_datetime('2025-05-16')
    df_pivot = df.pivot(index='Date', columns='ID', values='Free_Float_MktCap')
    return df_pivot

def load_volume_data(file_path, sheet_name='avg_vol'):
    """Load 6-month average volume data from Excel."""
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df['Date'] = pd.to_datetime('2025-05-16')
    df_pivot = df.pivot(index='Date', columns='ID', values='6M Average Volume')
    df_pivot = df_pivot.replace('#N/A', np.nan)
    return df_pivot

def load_nifty50_data(file_path, sheet_name='nifty50'):
    """Load Nifty 50 data to get 6-month average volume."""
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df['Date'] = pd.to_datetime('2025-05-16')
    df_pivot = df.pivot(index='Date', columns='ID', values='6M Average Volume')
    df_pivot = df_pivot.replace('#N/A', np.nan)
    return df_pivot

def filter_universe(ffmc_df, rebal_date, top_n, valid_stocks):
    """Filter stocks to top N by free-float market cap from valid stocks."""
    rebal_date = pd.to_datetime(rebal_date)
    ffmc = ffmc_df.loc[rebal_date]
    ffmc = ffmc[ffmc.index.intersection(valid_stocks)]
    if len(ffmc) < top_n:
        print(f"Warning: Only {len(ffmc)} stocks available for top {top_n} selection")
    return ffmc.nlargest(top_n).index

def calculate_volatility(prices_df, start_date, end_date):
    """Calculate annualized volatility based on daily log returns."""
    prices_df = prices_df.loc[start_date:end_date]
    log_returns = np.log(prices_df / prices_df.shift(1)).dropna()
    volatility = log_returns.std() * np.sqrt(252)
    return volatility

def calculate_momentum_scores(prices_df, rebal_date):
    """Calculate Normalized Momentum Score for Nifty200 Momentum 15."""
    rebal_date = pd.to_datetime(rebal_date)
    
    m1 = prices_df.index[prices_df.index <= rebal_date][-1]
    m7 = prices_df.index[prices_df.index <= rebal_date - timedelta(days=180)][-1]
    m13 = prices_df.index[prices_df.index <= rebal_date - timedelta(days=360)][-1]
    
    print(f"Selected dates: m1={m1}, m7={m7}, m13={m13}")
    print(f"Data availability: m1={prices_df.loc[m1].notna().sum()}, "
          f"m7={prices_df.loc[m7].notna().sum()}, m13={prices_df.loc[m13].notna().sum()} stocks")
    
    return_6m = (prices_df.loc[m1] / prices_df.loc[m7]) - 1
    return_12m = (prices_df.loc[m1] / prices_df.loc[m13]) - 1
    
    nan_12m = return_12m[return_12m.isna()].index.tolist()
    print(f"NaN in 6-month returns: {return_6m.isna().sum()} stocks")
    print(f"NaN in 12-month returns: {len(nan_12m)} stocks {nan_12m}")
    
    start_date = rebal_date - timedelta(days=365)
    log_returns = np.log(prices_df.loc[start_date:rebal_date] / prices_df.loc[start_date:rebal_date].shift(1)).dropna()
    sigma_p = log_returns.std() * np.sqrt(252)
    print(f"Valid volatility for {sigma_p.notna().sum()} stocks")
    
    mr6 = return_6m / sigma_p
    mr12 = return_12m / sigma_p
    
    mu_mr6 = mr6.mean()
    sigma_mr6 = mr6.std()
    mu_mr12 = mr12.mean()
    sigma_mr12 = mr12.std()
    
    z_mr6 = (mr6 - mu_mr6) / sigma_mr6
    z_mr12 = (mr12 - mu_mr12) / sigma_mr12
    
    weighted_z = 0.5 * z_mr6 + 0.5 * z_mr12
    
    normalized_score = np.where(weighted_z >= 0,
                               1 + weighted_z,
                               (1 - weighted_z) ** -1)
    
    scores = pd.Series(normalized_score, index=prices_df.columns)
    nan_scores = scores[scores.isna()].index.tolist()
    print(f"NaN in final momentum scores: {len(nan_scores)} stocks {nan_scores}")
    
    return scores

def calculate_low_vol_index(prices_df, ffmc_df, volume_df, nifty50_volume_df, rebal_date, top_n=30):
    """Construct Nifty100 Low Volatility 30 Index with turnover-based weight capping."""
    rebal_date = pd.to_datetime(rebal_date)
    start_date = rebal_date - timedelta(days=365)
    
    # Calculate volatility
    volatility = calculate_volatility(prices_df, start_date, rebal_date)
    
    # Filter stocks present in ffmc_df
    common_stocks = volatility.index.intersection(ffmc_df.columns)
    volatility = volatility[common_stocks]
    
    # Rank stocks by volatility (lowest first)
    vol_ranks = volatility.rank(ascending=True)
    
    # Select top N stocks
    selected_stocks = vol_ranks.nsmallest(top_n).index
    
    # Calculate weights based on inverse volatility
    selected_vol = volatility[selected_stocks]
    inverse_vol = 1 / selected_vol
    weights = inverse_vol / inverse_vol.sum()
    
    # Load 6-month average volume for selected stocks
    volume = volume_df.loc[rebal_date, selected_stocks].replace('#N/A', np.nan)
    
    # Get the minimum 6-month average volume from Nifty 50
    nifty50_volume = nifty50_volume_df.loc[rebal_date]
    min_nifty50_volume = nifty50_volume.min()
    print(f"Minimum Nifty 50 6-month average volume: {min_nifty50_volume}")
    
    # Identify stocks with volume less than the minimum Nifty 50 volume
    low_volume_stocks = volume < min_nifty50_volume
    print(f"Stocks with low volume (capped at 3%): {low_volume_stocks[low_volume_stocks].index.tolist()}")
    
    # Cap weights at 3% for low-volume stocks
    capped_weights = weights.copy()
    capped_weights[low_volume_stocks] = 0.03
    
    # Calculate excess weight and redistribute among non-capped stocks
    excess = (weights[low_volume_stocks] - 0.03).sum()
    non_capped = ~low_volume_stocks
    if non_capped.sum() > 0 and excess > 0:
        capped_weights[non_capped] += excess * (inverse_vol[non_capped] / inverse_vol[non_capped].sum())
    
    # Normalize weights to sum to 1
    capped_weights = capped_weights / capped_weights.sum()
    
    return pd.DataFrame({
        'Volatility': selected_vol,
        'Weight': capped_weights*100
    }), volatility

def calculate_momentum_index(prices_df, ffmc_df, rebal_date, top_n=15):
    """Construct Nifty200 Momentum 15 Index."""
    rebal_date = pd.to_datetime(rebal_date)
    
    momentum_scores = calculate_momentum_scores(prices_df, rebal_date)
    
    common_stocks = momentum_scores.index.intersection(ffmc_df.columns)
    momentum_scores = momentum_scores[common_stocks]
    
    momentum_scores = momentum_scores.dropna()
    if momentum_scores.empty:
        print("Warning: No valid momentum scores after filtering. Check data for selected dates.")
        return pd.DataFrame({'Momentum_Score': [], 'Weight': []}), momentum_scores
    
    selected_stocks = momentum_scores.nlargest(top_n).index
    
    ffmc = ffmc_df.loc[rebal_date, selected_stocks]
    
    scores = momentum_scores[selected_stocks]
    weights = ffmc * scores
    weights = weights / weights.sum()
    
    ffmc_weights = ffmc / ffmc.sum()
    cap_limit = np.minimum(0.05, 5 * ffmc_weights)
    excess = (weights - cap_limit).clip(lower=0).sum()
    weights = np.minimum(weights, cap_limit)
    
    uncapped = weights < cap_limit
    if uncapped.sum() > 0:
        weights[uncapped] = weights[uncapped] + (excess * weights[uncapped] / weights[uncapped].sum())
    
    weights = weights / weights.sum()
    
    return pd.DataFrame({
        'Momentum_Score': momentum_scores[selected_stocks],
        'Weight': weights*100
    }), momentum_scores

def main():
    file_path = 'factor_data.xlsx'
    
    try:
        prices = load_price_data(file_path, sheet_name='price', rebal_date='2025-05-16')
        ffmc = load_ffmc_data(file_path, sheet_name='maktcap')
        volume = load_volume_data(file_path, sheet_name='avg_vol')
        nifty50_volume = load_nifty50_data(file_path, sheet_name='nifty50')
    except Exception as e:
        print(f"Error loading data: {e}")
        return
    
    print("Available dates in price data:", prices.index.min(), "to", prices.index.max())
    print(f"Total stocks after filtering: {len(prices.columns)}")
    
    rebal_date = '2025-05-16'
    
    try:
        nifty100_stocks = filter_universe(ffmc, rebal_date, top_n=100, valid_stocks=prices.columns)
        nifty200_stocks = filter_universe(ffmc, rebal_date, top_n=200, valid_stocks=prices.columns)
    except Exception as e:
        print(f"Error filtering universe: {e}")
        return
    
    nifty100_prices = prices[nifty100_stocks]
    nifty200_prices = prices[nifty200_stocks]
    nifty100_ffmc = ffmc[nifty100_stocks]
    nifty200_ffmc = ffmc[nifty200_stocks]
    
    try:
        low_vol_index, all_vol_scores = calculate_low_vol_index(
            nifty100_prices, nifty100_ffmc, volume, nifty50_volume, rebal_date, top_n=30
        )
        momentum_index, all_momentum_scores = calculate_momentum_index(
            nifty200_prices, nifty200_ffmc, rebal_date, top_n=15
        )
        
        all_vol_scores.to_csv('nifty100_all_volatility_scores.csv')
        print(f"Saved volatility scores for {len(all_vol_scores)} Nifty 100 stocks to 'nifty100_all_volatility_scores.csv'")
        
        all_momentum_scores.to_csv('nifty200_all_momentum_scores.csv')
        print(f"Saved momentum scores for {len(all_momentum_scores)} Nifty 200 stocks to 'nifty200_all_momentum_scores.csv'")
        
        low_vol_index.to_csv('nifty100_low_volatility_30.csv')
        momentum_index.to_csv('nifty200_momentum_15.csv')
        
        print("Nifty100 Low Volatility 30 Index:")
        print(low_vol_index)
        print("\nNifty200 Momentum 15 Index:")
        print(momentum_index)
    except Exception as e:
        print(f"Error calculating indices or saving scores: {e}")
        return

if __name__ == "__main__":
    main()

Stocks with non-NaN prices on [Timestamp('2025-05-16 00:00:00'), Timestamp('2024-11-17 00:00:00'), Timestamp('2024-05-21 00:00:00')]: 480
Available dates in price data: 2023-01-01 00:00:00 to 2025-05-16 00:00:00
Total stocks after filtering: 480
Minimum Nifty 50 6-month average volume: 1572967863.93443
Stocks with low volume (capped at 3%): ['PIDI IS Equity', 'SRCM IS Equity', 'ICICIGI IS Equity', 'UNITDSPR IS Equity']
Selected dates: m1=2025-05-16 00:00:00, m7=2024-11-17 00:00:00, m13=2024-05-21 00:00:00
Data availability: m1=200, m7=200, m13=200 stocks
NaN in 6-month returns: 0 stocks
NaN in 12-month returns: 0 stocks []
Valid volatility for 200 stocks
NaN in final momentum scores: 0 stocks []
Saved volatility scores for 100 Nifty 100 stocks to 'nifty100_all_volatility_scores.csv'
Saved momentum scores for 200 Nifty 200 stocks to 'nifty200_all_momentum_scores.csv'
Nifty100 Low Volatility 30 Index:
                    Volatility    Weight
ITC IS Equity         0.152677  4.161718
NEST 

In [1]:
import dash
from dash import dcc, html, dash_table, Input, Output, callback
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

# Initialize the Dash app
app = dash.Dash(__name__)

# Load and process data
def load_and_process_data():
    low_vol_30 = pd.read_csv('nifty100_low_volatility_30.csv')
    momentum_15 = pd.read_csv('nifty200_momentum_15.csv')
    all_momentum = pd.read_csv('nifty200_all_momentum_scores.csv')
    all_volatility = pd.read_csv('nifty100_all_volatility_scores.csv')
    
    existing_index_file = pd.ExcelFile('existing_index.xlsx')
    existing_low_vol_df = pd.read_excel(existing_index_file, sheet_name='low vol')
    existing_momentum_df = pd.read_excel(existing_index_file, sheet_name='mom')
    
    mktcap_df = pd.read_excel('factor_data.xlsx', sheet_name='maktcap')
    name_to_id_mapping = dict(zip(mktcap_df['Name'], mktcap_df['ID']))
    
    manual_mappings = {
        'The Indian Hotels Company Ltd.': 'IH IS Equity',
        'Indian Hotels Company Ltd.': 'IH IS Equity',
        'Coforge Ltd.': 'COFORGE IS Equity',
        'Coforge Limited': 'COFORGE IS Equity',
        'HDFC Bank Ltd.': 'HDFCB IS Equity',
        'Titan Company Ltd.': 'TTAN IS Equity',
    }
    name_to_id_mapping.update(manual_mappings)
    
    return (low_vol_30, momentum_15, all_momentum, all_volatility, 
            existing_low_vol_df, existing_momentum_df, name_to_id_mapping)

def clean_company_name(name):
    name = str(name)
    name = (name.replace(' Ltd.', '').replace(' Limited', '').replace(' Ltd', '').replace(' LTD', '')
            .replace('.', '').replace('&', 'AND').replace(',', '').replace("'", '').strip())
    if name.upper().startswith('THE '):
        name = name[4:]
    return name.upper()

def convert_names_to_ids(company_names, name_to_id_mapping):
    ids = []
    unmapped_names = []
    manual_mappings = {
        'The Indian Hotels Company Ltd.': 'IH IS Equity',
        'Indian Hotels Company Ltd.': 'IH IS Equity',
        'Coforge Ltd.': 'COFORGE IS Equity',
        'Coforge Limited': 'COFORGE IS Equity',
        'HDFC Bank Ltd.': 'HDFCB IS Equity',
        'Titan Company Ltd.': 'TTAN IS Equity',
    }
    
    for name in company_names:
        if name in ['Tri-Party Repo (TREPS)', 'Cash & Cash Equivalent', 'Net Current Asset']:
            ids.append('N/A')
            continue
            
        matched = False
        matched_id = None
        clean_name = clean_company_name(name)
        
        if name in manual_mappings:
            matched_id = manual_mappings[name]
            matched = True
        elif clean_name in manual_mappings:
            matched_id = manual_mappings[clean_name]
            matched = True
        elif name in name_to_id_mapping:
            matched_id = name_to_id_mapping[name]
            matched = True
        elif clean_name in name_to_id_mapping:
            matched_id = name_to_id_mapping[clean_name]
            matched = True
        else:
            best_match = None
            best_score = 0
            
            for mapped_name, mapped_id in name_to_id_mapping.items():
                clean_mapped = clean_company_name(mapped_name)
                score = fuzz.ratio(clean_name, clean_mapped)
                
                if score > best_score and score > 80:
                    best_score = score
                    best_match = (mapped_name, mapped_id)
            
            if best_match:
                matched_id = best_match[1]
                matched = True
        
        if matched and matched_id:
            ids.append(matched_id)
        else:
            ids.append('UNMAPPED')
            unmapped_names.append(name)
    
    return ids

# Load data
(low_vol_30, momentum_15, all_momentum, all_volatility, 
 existing_low_vol_df, existing_momentum_df, name_to_id_mapping) = load_and_process_data()

# Convert names to IDs
existing_low_vol_df['ID'] = convert_names_to_ids(existing_low_vol_df['Company Name'].tolist(), name_to_id_mapping)
existing_momentum_df['ID'] = convert_names_to_ids(existing_momentum_df['Company Name'].tolist(), name_to_id_mapping)

# Filter out unmapped stocks
existing_low_vol_df = existing_low_vol_df[existing_low_vol_df['ID'] != 'UNMAPPED']
existing_momentum_df = existing_momentum_df[existing_momentum_df['ID'] != 'UNMAPPED']

# Calculate overlap data
def calculate_weighted_overlap_data(new_portfolio, existing_index_df):
    existing_index_df = existing_index_df[existing_index_df['ID'] != 'UNMAPPED']
    merged = pd.merge(new_portfolio, existing_index_df, 
                     left_on='Unnamed: 0', right_on='ID', how='inner')
    
    if not merged.empty:
        new_weights = merged['Weight']
        existing_weights = merged['Holding Percentage']
        merged['weight_overlap'] = np.minimum(new_weights, existing_weights)
        total_weight_overlap = merged['weight_overlap'].sum()
        
        overlap_count = len(merged)
        total_new = len(new_portfolio)
        
        merged['New_Weight'] = new_weights
        merged['Existing_Weight'] = existing_weights
        merged['Overlap_Weight'] = merged['weight_overlap']
        
        display_columns = ['Unnamed: 0', 'Company Name', 'New_Weight', 'Existing_Weight', 'Overlap_Weight']
        merged_display = merged[display_columns].sort_values('Overlap_Weight', ascending=False)
        
        return {
            'overlap_count': overlap_count,
            'total_new': total_new,
            'weight_overlap': total_weight_overlap,
            'merged_data': merged_display
        }
    return {'overlap_count': 0, 'total_new': len(new_portfolio), 'weight_overlap': 0, 'merged_data': pd.DataFrame()}

# Calculate quartile data - MODIFIED
def calculate_quartile_data(index_df, factor_scores_df, factor_name):
    factor_scores = factor_scores_df.set_index('Unnamed: 0')[factor_scores_df.columns[1]]
    quartiles = pd.qcut(factor_scores, q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    index_quartiles = index_df['ID'].map(quartiles).value_counts().reindex(['Q1', 'Q2', 'Q3', 'Q4'], fill_value=0)
    
    total = len(index_df)
    quartile_pct = (index_quartiles / total * 100).round(1)
    
    index_with_scores = pd.merge(index_df, factor_scores.rename('factor_score'), 
                                left_on='ID', right_index=True, how='left')
    
    # Add quartile column
    index_with_scores['quartile'] = index_with_scores['ID'].map(quartiles)
    
    # Sort for full list
    if factor_name == 'Low Volatility':
        all_stocks_sorted = index_with_scores.sort_values(by=['quartile', 'factor_score'], ascending=[True, True])
    else:  # Momentum
        all_stocks_sorted = index_with_scores.sort_values(by=['quartile', 'factor_score'], ascending=[True, False])
    
    all_stocks_with_quartile = all_stocks_sorted[['ID', 'Company Name', 'factor_score', 'quartile']]
    
    return {
        'quartile_pct': quartile_pct,
        'top_stocks': index_with_scores.sort_values('factor_score', ascending=(factor_name == 'Low Volatility')).head(5),
        'bottom_stocks': index_with_scores.sort_values('factor_score', ascending=(factor_name != 'Low Volatility')).head(5),
        'all_scores': index_with_scores['factor_score'].dropna(),
        'universe_scores': factor_scores,
        'all_stocks_with_quartile': all_stocks_with_quartile
    }

# Calculate missing stocks analysis
def calculate_missing_stocks_analysis(new_portfolio, existing_index_df, factor_scores_df, factor_name):
    factor_scores = factor_scores_df.set_index('Unnamed: 0')[factor_scores_df.columns[1]]
    quartiles = pd.qcut(factor_scores, q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    
    new_portfolio_ids = set(new_portfolio['Unnamed: 0'].tolist())
    existing_index_ids = set(existing_index_df['ID'].tolist())
    
    missing_ids = existing_index_ids - new_portfolio_ids
    
    if missing_ids:
        missing_stocks_df = existing_index_df[existing_index_df['ID'].isin(missing_ids)].copy()
        missing_stocks_df['factor_score'] = missing_stocks_df['ID'].map(factor_scores)
        missing_stocks_df['quartile'] = missing_stocks_df['ID'].map(quartiles)
        
        missing_quartile_dist = missing_stocks_df['quartile'].value_counts().reindex(
            ['Q1', 'Q2', 'Q3', 'Q4'], fill_value=0
        )
        missing_quartile_pct = (missing_quartile_dist / len(missing_stocks_df) * 100).round(1)
        
        missing_stocks_sorted = missing_stocks_df.sort_values(
            'factor_score', 
            ascending=(factor_name == 'Low Volatility')
        )
        
        return {
            'missing_count': len(missing_stocks_df),
            'total_existing': len(existing_index_df),
            'missing_quartile_dist': missing_quartile_dist,
            'missing_quartile_pct': missing_quartile_pct,
            'missing_stocks_detailed': missing_stocks_sorted,
            'worst_misses': missing_stocks_sorted,
            'factor_name': factor_name
        }
    
    return {
        'missing_count': 0,
        'total_existing': len(existing_index_df),
        'missing_quartile_dist': pd.Series([0, 0, 0, 0], index=['Q1', 'Q2', 'Q3', 'Q4']),
        'missing_quartile_pct': pd.Series([0, 0, 0, 0], index=['Q1', 'Q2', 'Q3', 'Q4']),
        'missing_stocks_detailed': pd.DataFrame(),
        'worst_misses': pd.DataFrame(),
        'factor_name': factor_name
    }

# Calculate all data
low_vol_overlap_data = calculate_weighted_overlap_data(low_vol_30, existing_low_vol_df)
momentum_overlap_data = calculate_weighted_overlap_data(momentum_15, existing_momentum_df)

low_vol_quartile_data = calculate_quartile_data(existing_low_vol_df, all_volatility, 'Low Volatility')
momentum_quartile_data = calculate_quartile_data(existing_momentum_df, all_momentum, 'Momentum')

low_vol_missing_data = calculate_missing_stocks_analysis(low_vol_30, existing_low_vol_df, all_volatility, 'Low Volatility')
momentum_missing_data = calculate_missing_stocks_analysis(momentum_15, existing_momentum_df, all_momentum, 'Momentum')

# Dashboard Layout
app.layout = html.Div([
    html.Div([
        html.H1("Portfolio Analysis Dashboard", className="header-title"),
        html.P("Interactive analysis of Low Volatility and Momentum portfolios", className="header-subtitle")
    ], className="header"),
    
    dcc.Tabs(id="main-tabs", value="overlap", children=[
        dcc.Tab(label="Portfolio Overlap Analysis", value="overlap"),
        dcc.Tab(label="Quartile Distribution", value="quartile"),
        dcc.Tab(label="Factor Score Analysis", value="factor"),
        dcc.Tab(label="Missing Stocks Analysis", value="missing")
    ]),
    
    html.Div(id="tab-content")
])

def create_overlap_table_columns(merged_data):
    columns = [
        {"name": "Stock ID", "id": "Unnamed: 0"},
        {"name": "Company Name", "id": "Company Name"},
        {"name": "New Weight (%)", "id": "New_Weight", "type": "numeric", "format": {"specifier": ".2f"}},
        {"name": "Existing Weight (%)", "id": "Existing_Weight", "type": "numeric", "format": {"specifier": ".2f"}},
        {"name": "Overlap Weight (%)", "id": "Overlap_Weight", "type": "numeric", "format": {"specifier": ".2f"}}
    ]
    return columns

def create_missing_stocks_table_columns():
    return [
        {"name": "Stock ID", "id": "ID"},
        {"name": "Company Name", "id": "Company Name"},
        {"name": "Previous Weight (%)", "id": "Holding Percentage", "type": "numeric", "format": {"specifier": ".2f"}},
        {"name": "Factor Score", "id": "factor_score", "type": "numeric", "format": {"specifier": ".4f"}},
        {"name": "Quartile", "id": "quartile"}
    ]

@callback(Output("tab-content", "children"), Input("main-tabs", "value"))
def render_tab_content(active_tab):
    if active_tab == "overlap":
        return html.Div([
            html.Div([
                html.Div([
                    html.H3("Low Volatility Portfolio Overlap"),
                    html.Div([
                        html.Div([
                            html.H4(f"{low_vol_overlap_data['overlap_count']}", className="metric-value"),
                            html.P("Overlapping Stocks", className="metric-label")
                        ], className="metric-card"),
                        html.Div([
                            html.H4(f"{low_vol_overlap_data['overlap_count']/low_vol_overlap_data['total_new']:.1%}", className="metric-value"),
                            html.P("Stock Overlap %", className="metric-label")
                        ], className="metric-card"),
                        html.Div([
                            html.H4(f"{low_vol_overlap_data['weight_overlap']:.1f}%", className="metric-value"),
                            html.P("Weight Overlap", className="metric-label")
                        ], className="metric-card")
                    ], className="metrics-row"),
                    
                    html.H4("Top Overlapping Stocks by Weight"),
                    dash_table.DataTable(
                        data=low_vol_overlap_data['merged_data'].head(10).to_dict('records'),
                        columns=create_overlap_table_columns(low_vol_overlap_data['merged_data']),
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
                        style_data_conditional=[
                            {
                                'if': {'row_index': 0},
                                'backgroundColor': '#e8f5e8'
                            }
                        ]
                    )
                ], className="portfolio-section"),
                
                html.Div([
                    html.H3("Momentum Portfolio Overlap"),
                    html.Div([
                        html.Div([
                            html.H4(f"{momentum_overlap_data['overlap_count']}", className="metric-value"),
                            html.P("Overlapping Stocks", className="metric-label")
                        ], className="metric-card"),
                        html.Div([
                            html.H4(f"{momentum_overlap_data['overlap_count']/momentum_overlap_data['total_new']:.1%}", className="metric-value"),
                            html.P("Stock Overlap %", className="metric-label")
                        ], className="metric-card"),
                        html.Div([
                            html.H4(f"{momentum_overlap_data['weight_overlap']:.1f}%", className="metric-value"),
                            html.P("Weight Overlap", className="metric-label")
                        ], className="metric-card")
                    ], className="metrics-row"),
                    
                    html.H4("Top Overlapping Stocks by Weight"),
                    dash_table.DataTable(
                        data=momentum_overlap_data['merged_data'].head(10).to_dict('records'),
                        columns=create_overlap_table_columns(momentum_overlap_data['merged_data']),
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
                        style_data_conditional=[
                            {
                                'if': {'row_index': 0},
                                'backgroundColor': '#e8f5e8'
                            }
                        ]
                    )
                ], className="portfolio-section")
            ], className="overlap-container")
        ])
    
    elif active_tab == "quartile":
        low_vol_quartile_fig = px.bar(
            x=['Q1 (Top)', 'Q2', 'Q3', 'Q4 (Bottom)'],
            y=low_vol_quartile_data['quartile_pct'].values,
            title="Low Volatility Index - Quartile Distribution",
            labels={'x': 'Quartile', 'y': 'Percentage (%)'},
            color=low_vol_quartile_data['quartile_pct'].values,
            color_continuous_scale='RdYlBu_r'
        )
        low_vol_quartile_fig.update_layout(showlegend=False, height=400)
        
        momentum_quartile_fig = px.bar(
            x=['Q1 (Top)', 'Q2', 'Q3', 'Q4 (Bottom)'],
            y=momentum_quartile_data['quartile_pct'].values,
            title="Momentum Index - Quartile Distribution",
            labels={'x': 'Quartile', 'y': 'Percentage (%)'},
            color=momentum_quartile_data['quartile_pct'].values,
            color_continuous_scale='RdYlBu_r'
        )
        momentum_quartile_fig.update_layout(showlegend=False, height=400)
        
        return html.Div([
            html.Div([
                html.Div([
                    dcc.Graph(figure=low_vol_quartile_fig)
                ], className="chart-container"),
                html.Div([
                    dcc.Graph(figure=momentum_quartile_fig)
                ], className="chart-container")
            ], className="charts-row"),
            
            html.Div([
                html.Div([
                    html.H4("Low Vol - Top 5 Stocks by Factor Score"),
                    dash_table.DataTable(
                        data=low_vol_quartile_data['top_stocks'][['Company Name', 'factor_score']].to_dict('records'),
                        columns=[
                            {"name": "Company Name", "id": "Company Name"},
                            {"name": "Factor Score", "id": "factor_score", "type": "numeric", "format": {"specifier": ".4f"}}
                        ],
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'}
                    )
                ], className="table-container"),
                
                html.Div([
                    html.H4("Momentum - Top 5 Stocks by Factor Score"),
                    dash_table.DataTable(
                        data=momentum_quartile_data['top_stocks'][['Company Name', 'factor_score']].to_dict('records'),
                        columns=[
                            {"name": "Company Name", "id": "Company Name"},
                            {"name": "Factor Score", "id": "factor_score", "type": "numeric", "format": {"specifier": ".4f"}}
                        ],
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'}
                    )
                ], className="table-container")
            ], className="tables-row"),
            
            html.Div([
                html.H3("Complete List of Stocks by Quartile"),
                html.Div([
                    html.H4("Low Volatility"),
                    dash_table.DataTable(
                        id='low-vol-table',
                        columns=[
                            {"name": "Stock ID", "id": "ID"},
                            {"name": "Company Name", "id": "Company Name"},
                            {"name": "Factor Score", "id": "factor_score", "type": "numeric", "format": {"specifier": ".4f"}},
                            {"name": "Quartile", "id": "quartile"}
                        ],
                        data=low_vol_quartile_data['all_stocks_with_quartile'].to_dict('records'),
                        filter_action='native',
                        sort_action='native',
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
                        style_data_conditional=[
                            {
                                'if': {'filter_query': '{quartile} = Q1'},
                                'backgroundColor': '#e8f5e8',
                                'color': 'black'
                            },
                            {
                                'if': {'filter_query': '{quartile} = Q4'},
                                'backgroundColor': '#ffebee',
                                'color': 'black'
                            }
                        ],
                        page_size=10
                    )
                ], className="table-container"),
                
                html.Div([
                    html.H4("Momentum"),
                    dash_table.DataTable(
                        id='momentum-table',
                        columns=[
                            {"name": "Stock ID", "id": "ID"},
                            {"name": "Company Name", "id": "Company Name"},
                            {"name": "Factor Score", "id": "factor_score", "type": "numeric", "format": {"specifier": ".4f"}},
                            {"name": "Quartile", "id": "quartile"}
                        ],
                        data=momentum_quartile_data['all_stocks_with_quartile'].to_dict('records'),
                        filter_action='native',
                        sort_action='native',
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
                        style_data_conditional=[
                            {
                                'if': {'filter_query': '{quartile} = Q1'},
                                'backgroundColor': '#e8f5e8',
                                'color': 'black'
                            },
                            {
                                'if': {'filter_query': '{quartile} = Q4'},
                                'backgroundColor': '#ffebee',
                                'color': 'black'
                            }
                        ],
                        page_size=10
                    )
                ], className="table-container")
            ], className="tables-row")
        ])
    
    elif active_tab == "factor":
        low_vol_dist_fig = go.Figure()
        low_vol_dist_fig.add_trace(go.Histogram(
            x=low_vol_quartile_data['universe_scores'],
            name='Full Universe',
            opacity=0.5,
            nbinsx=30
        ))
        low_vol_dist_fig.add_trace(go.Histogram(
            x=low_vol_quartile_data['all_scores'],
            name='Index Constituents',
            opacity=0.8,
            nbinsx=15
        ))
        low_vol_dist_fig.add_vline(
            x=low_vol_quartile_data['universe_scores'].quantile(0.75),
            line_dash="dash",
            line_color="blue",
            annotation_text="Q1 Boundary"
        )
        low_vol_dist_fig.add_vline(
            x=low_vol_quartile_data['universe_scores'].quantile(0.25),
            line_dash="dash",
            line_color="red",
            annotation_text="Q3 Boundary"
        )
        low_vol_dist_fig.update_layout(
            title="Low Volatility Factor Score Distribution",
            xaxis_title="Factor Score",
            yaxis_title="Number of Stocks",
            height=500
        )
        
        momentum_dist_fig = go.Figure()
        momentum_dist_fig.add_trace(go.Histogram(
            x=momentum_quartile_data['universe_scores'],
            name='Full Universe',
            opacity=0.5,
            nbinsx=30
        ))
        momentum_dist_fig.add_trace(go.Histogram(
            x=momentum_quartile_data['all_scores'],
            name='Index Constituents',
            opacity=0.8,
            nbinsx=15
        ))
        momentum_dist_fig.add_vline(
            x=momentum_quartile_data['universe_scores'].quantile(0.75),
            line_dash="dash",
            line_color="blue",
            annotation_text="Q1 Boundary"
        )
        momentum_dist_fig.add_vline(
            x=momentum_quartile_data['universe_scores'].quantile(0.25),
            line_dash="dash",
            line_color="red",
            annotation_text="Q3 Boundary"
        )
        momentum_dist_fig.update_layout(
            title="Momentum Factor Score Distribution",
            xaxis_title="Factor Score",
            yaxis_title="Number of Stocks",
            height=500
        )
        
        return html.Div([
            html.Div([
                dcc.Graph(figure=low_vol_dist_fig)
            ], className="chart-container"),
            html.Div([
                dcc.Graph(figure=momentum_dist_fig)
            ], className="chart-container")
        ])
    
    elif active_tab == "missing":
        low_vol_missing_fig = px.bar(
            x=['Q1 (Top)', 'Q2', 'Q3', 'Q4 (Bottom)'],
            y=low_vol_missing_data['missing_quartile_pct'].values,
            title="Low Volatility - Missing Stocks Quartile Distribution",
            labels={'x': 'Quartile', 'y': 'Percentage of Missing Stocks (%)'},
            color=low_vol_missing_data['missing_quartile_pct'].values,
            color_continuous_scale='Reds'
        )
        low_vol_missing_fig.update_layout(showlegend=False, height=400)
        
        momentum_missing_fig = px.bar(
            x=['Q1 (Top)', 'Q2', 'Q3', 'Q4 (Bottom)'],
            y=momentum_missing_data['missing_quartile_pct'].values,
            title="Momentum - Missing Stocks Quartile Distribution",
            labels={'x': 'Quartile', 'y': 'Percentage of Missing Stocks (%)'},
            color=momentum_missing_data['missing_quartile_pct'].values,
            color_continuous_scale='Reds'
        )
        momentum_missing_fig.update_layout(showlegend=False, height=400)
        
        return html.Div([
            html.Div([
                html.Div([
                    html.H3("Low Volatility - Missing Stocks Summary"),
                    html.Div([
                        html.Div([
                            html.H4(f"{low_vol_missing_data['missing_count']}", className="metric-value"),
                            html.P("Stocks Dropped", className="metric-label")
                        ], className="metric-card-red"),
                        html.Div([
                            html.H4(f"{low_vol_missing_data['missing_count']/low_vol_missing_data['total_existing']:.1%}", className="metric-value"),
                            html.P("Turnover Rate", className="metric-label")
                        ], className="metric-card-red"),
                        html.Div([
                            html.H4(f"{low_vol_missing_data['missing_quartile_dist']['Q1']}", className="metric-value"),
                            html.P("Q1 Stocks Lost", className="metric-label")
                        ], className="metric-card-red")
                    ], className="metrics-row")
                ], className="portfolio-section"),
                
                html.Div([
                    html.H3("Momentum - Missing Stocks Summary"),
                    html.Div([
                        html.Div([
                            html.H4(f"{momentum_missing_data['missing_count']}", className="metric-value"),
                            html.P("Stocks Dropped", className="metric-label")
                        ], className="metric-card-red"),
                        html.Div([
                            html.H4(f"{momentum_missing_data['missing_count']/momentum_missing_data['total_existing']:.1%}", className="metric-value"),
                            html.P("Turnover Rate", className="metric-label")
                        ], className="metric-card-red"),
                        html.Div([
                            html.H4(f"{momentum_missing_data['missing_quartile_dist']['Q1']}", className="metric-value"),
                            html.P("Q1 Stocks Lost", className="metric-label")
                        ], className="metric-card-red")
                    ], className="metrics-row")
                ], className="portfolio-section")
            ], className="overlap-container"),
            
            html.Div([
                html.Div([
                    dcc.Graph(figure=low_vol_missing_fig)
                ], className="chart-container"),
                html.Div([
                    dcc.Graph(figure=momentum_missing_fig)
                ], className="chart-container")
            ], className="charts-row"),
            
            html.Div([
                html.Div([
                    html.H4("Low Vol - Top 10 Best Performing Stocks That Were Dropped"),
                    dash_table.DataTable(
                        data=low_vol_missing_data['worst_misses'][['ID', 'Company Name', 'Holding Percentage', 'factor_score', 'quartile']].to_dict('records'),
                        columns=create_missing_stocks_table_columns(),
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
                        style_data_conditional=[
                            {
                                'if': {'filter_query': '{quartile} = Q1'},
                                'backgroundColor': '#ffebee',
                                'color': 'black',
                            }
                        ]
                    )
                ], className="table-container"),
                
                html.Div([
                    html.H4("Momentum - Top 10 Best Performing Stocks That Were Dropped"),
                    dash_table.DataTable(
                        data=momentum_missing_data['worst_misses'][['ID', 'Company Name', 'Holding Percentage', 'factor_score', 'quartile']].to_dict('records'),
                        columns=create_missing_stocks_table_columns(),
                        style_cell={'textAlign': 'left', 'padding': '10px'},
                        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
                        style_data_conditional=[
                            {
                                'if': {'filter_query': '{quartile} = Q1'},
                                'backgroundColor': '#ffebee',
                                'color': 'black',
                            }
                        ]
                    )
                ], className="table-container")
            ], className="tables-row")
        ])

# Custom CSS styling
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            body {
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                margin: 0;
                padding: 0;
                background-color: #f8f9fa;
            }
            .header {
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                padding: 2rem;
                text-align: center;
                margin-bottom: 2rem;
            }
            .header-title {
                margin: 0;
                font-size: 2.5rem;
                font-weight: 300;
            }
            .header-subtitle {
                margin: 0.5rem 0 0 0;
                font-size: 1.2rem;
                opacity: 0.9;
            }
            .portfolio-section {
                background: white;
                border-radius: 10px;
                padding: 2rem;
                margin: 1rem;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }
            .metrics-row {
                display: flex;
                gap: 1rem;
                margin: 1rem 0 2rem 0;
            }
            .metric-card {
                flex: 1;
                background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
                color: white;
                padding: 1.5rem;
                border-radius: 10px;
                text-align: center;
            }
            .metric-card-red {
                flex: 1;
                background: linear-gradient(135deg, #ff9a9e 0%, #fecfef 100%);
                color: #333;
                padding: 1.5rem;
                border-radius: 10px;
                text-align: center;
            }
            .metric-value {
                font-size: 2rem;
                font-weight: bold;
                margin: 0;
            }
            .metric-label {
                margin: 0.5rem 0 0 0;
                opacity: 0.9;
            }
            .overlap-container {
                display: flex;
                flex-direction: column;
                gap: 2rem;
            }
            .charts-row {
                display: flex;
                gap: 2rem;
                margin: 2rem 0;
            }
            .chart-container {
                flex: 1;
                background: white;
                border-radius: 10px;
                padding: 1rem;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }
            .tables-row {
                display: flex;
                gap: 2rem;
                margin: 2rem 0;
            }
            .table-container {
                flex: 1;
                background: white;
                border-radius: 10px;
                padding: 1.5rem;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }
            .dash-table-container .dash-spreadsheet-container .dash-spreadsheet-inner {
                border-radius: 10px !important;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

if __name__ == '__main__':
    app.run(debug=True, port=8050)

c:\Users\Harsha.1Aditya\AppData\Local\Programs\Python\Python313\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
